# K Nearest Neighbors

In [14]:
import numpy as np
import pandas as pd
import operator

leaf = pd.read_csv('train.csv')

In [15]:
print(leaf)

       id                       species   margin1   margin2   margin3  \
0       1                   Acer_Opalus  0.007812  0.023438  0.023438   
1       2         Pterocarya_Stenoptera  0.005859  0.000000  0.031250   
2       3          Quercus_Hartwissiana  0.005859  0.009766  0.019531   
3       5               Tilia_Tomentosa  0.000000  0.003906  0.023438   
4       6            Quercus_Variabilis  0.005859  0.003906  0.048828   
5       8          Magnolia_Salicifolia  0.070312  0.093750  0.033203   
6      10           Quercus_Canariensis  0.021484  0.031250  0.017578   
7      11                 Quercus_Rubra  0.000000  0.000000  0.037109   
8      14               Quercus_Brantii  0.005859  0.001953  0.033203   
9      15                Salix_Fragilis  0.000000  0.000000  0.009766   
10     17               Zelkova_Serrata  0.019531  0.031250  0.001953   
11     18         Betula_Austrosinensis  0.001953  0.001953  0.023438   
12     20               Quercus_Pontica  0.015625  

In [16]:
allData = leaf.drop(columns = ['id', 'species'])
trainLeaf, validLeaf, testLeaf = allData.iloc[0:600], allData.iloc[601:800], allData.iloc[801:989]
col = np.arange(0,leaf.shape[1])
col = np.delete(col, 1)
#print(allData)
#names = allData.drop(leaf.columns[col], axis = 1)
#print(names)

In [17]:
print(validLeaf.shape)

(199, 192)


In [18]:
def ptDist(pt1, pt2, length):
    distance = 0
    i = 0
    while i < length:
        distance = distance + np.square(pt1[i] - pt2[i])
        i=i+1
    return distance

def getLeaf(ind):
    return trainLeaf.iloc[ind]


In [19]:
ptDist(getLeaf(26),getLeaf(30), 192)

0.19464171182654844

In [20]:
def knn(subject, k):
    distances = {}
    sortedOrder = {}
    #calculate the distance to every data point in trainleaf, our dataset
    for x in range(len(trainLeaf)):
        distances[x] = ptDist(subject, getLeaf(x), 192)
    sortedOrder = sorted(distances.items(), key = operator.itemgetter(1)) # sorted by distance, tuples of index and distance
    #getting k nearest neighbors
    neighbors = []
    for y in range(k):
        neighbors.append(sortedOrder[y])
    #see if the neighbors have a majority approval
    possibleSpecies = {}
    for z in range(len(neighbors)):
        newSpec = leaf.iloc[int(neighbors[z][0])][1]#for the nieghbor z, we get its original index
        if newSpec in possibleSpecies:
            possibleSpecies[newSpec] = possibleSpecies[newSpec]+1
        else:
            possibleSpecies[newSpec] = 1
    bestGuess = max(possibleSpecies.items(), key = operator.itemgetter(1))[0]
    return bestGuess#we now have all the distances, in the order of the points
knn(getLeaf(10), 10)

'Zelkova_Serrata'

In [21]:
#Quick test on the Training data, should be perfect(since each point will find itself)
suc = 0
total = 0
k = 3
for i in range(10):
    res = knn(getLeaf(i), k)
    print(res)
    if res == leaf.iloc[i][1]:
        suc += 1
    total += 1
print(100* (suc / total))

Acer_Opalus
Pterocarya_Stenoptera
Quercus_Hartwissiana
Tilia_Tomentosa
Quercus_Variabilis
Magnolia_Salicifolia
Quercus_Canariensis
Quercus_Rubra
Quercus_Brantii
Salix_Fragilis
100.0


In [24]:
#Test over validation data to determine the correct k to use
def getValidLeaf(ind):
    return validLeaf.iloc[ind]

for k in range(1,20):
    suc = 0
    total = 0
    for i in range(len(validLeaf)):
        res = knn(getValidLeaf(i), k)
        #print(res)
        if res == leaf.iloc[i+601][1]:
            suc += 1
        total += 1
    print("With k= "+str(k)+ "success rate was: " + str(100 *(float(suc) / float(total))))

With k= 1success rate was: 88.94472361809045
With k= 2success rate was: 88.94472361809045
With k= 3success rate was: 87.43718592964824
With k= 4success rate was: 87.93969849246231
With k= 5success rate was: 86.4321608040201
With k= 6success rate was: 86.4321608040201
With k= 7success rate was: 82.91457286432161
With k= 8success rate was: 77.88944723618091
With k= 9success rate was: 78.39195979899498
With k= 10success rate was: 77.38693467336684
With k= 11success rate was: 77.38693467336684
With k= 12success rate was: 75.37688442211056
With k= 13success rate was: 72.8643216080402
With k= 14success rate was: 70.35175879396985
With k= 15success rate was: 68.84422110552764
With k= 16success rate was: 66.83417085427136
With k= 17success rate was: 63.81909547738693
With k= 18success rate was: 63.31658291457286
With k= 19success rate was: 62.311557788944725


In [22]:
#Just making sure that the training data spans the full set of possible results
names = []
for i in range(600):
    names.append(leaf.iloc[i][1])

for i in range(len(validLeaf)):
    actual = leaf.iloc[i+601][1]
    if not(actual in names):
        print(actual + "not in set")
for i in range(len(testLeaf)):
    actual = leaf.iloc[i+801][1]
    if not(actual in names):
        print(actual + "not in set")

In [23]:
#Test on the Testing data:

def getTestLeaf(ind):
    return testLeaf.iloc[ind]
suc = 0
total = 0
k = 1

for i in range(len(testLeaf)):
    res = knn(getTestLeaf(i), k)
    #print(res)
    if res == leaf.iloc[i+801][1]:
        suc += 1
    total += 1
    print(suc / total)
print(100 *(float(suc) / float(total)))
# Initial results, with k = 3, indicates 87.47% correct

0.0
0.5
0.6666666666666666
0.75
0.8
0.8333333333333334
0.8571428571428571
0.875
0.8888888888888888
0.9
0.9090909090909091
0.9166666666666666
0.9230769230769231
0.9285714285714286
0.9333333333333333
0.9375
0.9411764705882353
0.9444444444444444
0.8947368421052632
0.9
0.9047619047619048
0.9090909090909091
0.9130434782608695
0.9166666666666666
0.88
0.8846153846153846
0.8888888888888888
0.8571428571428571
0.8275862068965517
0.8333333333333334
0.8387096774193549
0.84375
0.8181818181818182
0.8235294117647058
0.8285714285714286
0.8333333333333334
0.8378378378378378
0.8421052631578947
0.8461538461538461
0.85
0.8536585365853658
0.8571428571428571
0.8604651162790697
0.8636363636363636
0.8666666666666667
0.8695652173913043
0.8723404255319149
0.875
0.8571428571428571
0.86
0.8627450980392157
0.8653846153846154
0.8679245283018868
0.8703703703703703
0.8727272727272727
0.875
0.8771929824561403
0.8793103448275862
0.8813559322033898
0.8833333333333333
0.8688524590163934
0.8709677419354839
0.8730158730158

Result of test: 90.42553191489363% success.